In [1]:
### kaggle

In [3]:
import pandas as pd
import numpy as np
from nltk.tokenize import sent_tokenize
import en_coref_lg
nlp = en_coref_lg.load()
from dateutil import parser
import re

In [4]:
def summarise_fast(article, n=3):
    """"""
    # resolve co-reference issues
    clusters = nlp(article)._.coref_clusters

    if isinstance(clusters, type(None)):
        trans_dict = {}
    else:
        clusters = filter(None, clusters)
        trans_dict = {str(i[1]): str(i[0]) for i in clusters}

    for k, v in trans_dict.items():
        article = article.replace(k, v)

    return article

In [244]:
## Kaggle Dataset (Fake News)

In [523]:
df_fake = pd.read_csv('../../data/external/getting-real-fn/fake.csv')

In [524]:
df_fake = df_fake[df_fake.type.isin(['bias', 'conspiracy', 'fake', 'hate', 'junksci', 'satire', 'state'])]

In [525]:
df_fake = df_fake[df_fake['language']=='english']

In [526]:
df_fake['ym'] = df_fake['published'].apply(lambda x: x[:7])

In [527]:
df_fake = df_fake.reset_index()

In [528]:
df_fake = df_fake.dropna(subset=['text', 'title'])

In [529]:
df_fake['triple_text'] = df_fake[['title', 'text']].apply(lambda x: str(x['title'])+". "+" ".join(sent_tokenize(str(x['text']))[:2]).replace('\n',''), 1)

In [558]:
df_fake['triple_text'].iloc[717]

'The Email That Got John Podesta Hacked. The Email That Got John Podesta Hacked Latest WikiLeaks dump reveals phishing email that compromised Podesta Mikael Thalen - October 28, 2016 Comments The email that lead to the hack of Clinton campaign chairman John Podesta’s Gmail account was published online Friday by WikiLeaks. The malicious March 2016 email, which claims to be from Google, asked Podesta to change his password due to unauthorized access from an IP address in the Ukraine.'

In [553]:
np.where(df_fake['triple_text'].str.contains('.com', case=True))

(array([   3,    4,    6,    7,    8,    9,   11,   12,   13,   14,   16,
          20,   26,   27,   30,   34,   37,   47,   54,   57,   73,   87,
          88,   97,   99,  117,  131,  164,  174,  175,  176,  177,  178,
         182,  184,  186,  187,  189,  190,  191,  192,  193,  194,  195,
         198,  200,  204,  206,  215,  217,  221,  227,  233,  235,  238,
         244,  245,  249,  252,  253,  254,  258,  261,  263,  264,  267,
         269,  271,  273,  274,  288,  289,  290,  296,  298,  304,  307,
         309,  315,  324,  326,  334,  350,  360,  369,  370,  381,  385,
         386,  389,  390,  392,  394,  403,  405,  416,  421,  426,  437,
         440,  441,  451,  453,  456,  461,  462,  466,  475,  481,  483,
         491,  494,  500,  501,  508,  514,  515,  516,  517,  523,  524,
         526,  534,  543,  549,  550,  555,  557,  560,  563,  564,  568,
         578,  583,  592,  608,  612,  617,  619,  625,  631,  632,  636,
         639,  643,  644,  646,  651, 

In [536]:
df_fake['triple_text_clean'] = df_fake['triple_text'].apply(lambda x: re.sub('.com$', '', x))

In [544]:
np.sum(df_fake['triple_text'] == df_fake['triple_text_clean'])

1350

In [458]:
df_fake['coref_triples'] = df_fake['triple_text'].apply(lambda x: summarise_fast(x))

In [401]:
df_fake['fake'] = 1

In [402]:
df_fake.to_csv('../../data/external/getting-real-fn/fake_triples.csv')

In [1]:
## Isot-Dataset (Fake News)

In [170]:
df[(df.text.str.contains('Robert De Niro')) & (df.text.str.contains('Trump'))]

,title,text,subject,date
218,"Robert De Niro Attends Charity Event, Tears I...",Acclaimed actor Robert De Niro doesn t hold ba...,News,"October 16, 2017"
2934,"Alec Baldwin Joins Anti-Trump Protest, Gives ...","Over the next few days, America (and the world...",News,"January 20, 2017"
4307,Robert De Niro Just Told The World He Wants T...,Robert De Niro was supposed to appear in a non...,News,"October 8, 2016"
5070,WATCH: Actor Robert De Niro SLAMS ‘Totally Nu...,Legendary actor Robert De Niro was in Sarajevo...,News,"August 13, 2016"
6368,WATCH: Robert De Niro Makes Sure To Perfectly...,Every year GLAAD holds their media awards cere...,News,"May 15, 2016"
12425,ROBERT DENIRO Wanted “To Punch Trump In The Fa...,Robert De Niro gave anti-Donald Trump proteste...,politics,"Nov 12, 2016"
19635,ROBERT DENIRO Wanted “To Punch Trump In The Fa...,Robert De Niro gave anti-Donald Trump proteste...,left-news,"Nov 12, 2016"


In [159]:
df = pd.read_csv('../../data/external/isot_dataset/Fake.csv')

In [49]:
df.head()

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [50]:
df.subject.unique()

array(['News', 'politics', 'Government News', 'left-news', 'US_News',
       'Middle-east'], dtype=object)

In [51]:
df_case = df[df.subject.isin(['politics', 'US_News', 'Government News'])]

In [52]:
df_case = df_case[~df_case.date.str.contains('https')]

In [53]:
df_case['ym'] = df_case['date'].apply(lambda x: parser.parse(x)).astype(str)

In [54]:
df_case = df_case[df_case.ym.str[:7].isin(['2016-07','2016-08','2016-09','2016-10','2016-11','2016-12'])]

In [55]:
keys = ['trump', 'hillary', 'election', 'obama']

In [56]:
t_arrays = []
for i in keys:
    t_arrays.extend(np.where(df_case['text'].str.contains(i, case=False))[0])

In [57]:
df_filter = df_case.iloc[np.array(list(set(t_arrays)))]

In [58]:
df_filter = df_filter.dropna(subset=['text', 'title'])

In [59]:
df_filter['fake'] = 1

In [60]:
len(df_filter)

1428

In [114]:
df_filter = df_filter.reset_index()

In [115]:
df_filter['triple_text'] = df_filter[['title', 'text']].apply(lambda x: str(x['title'])+". "+" ".join(sent_tokenize(x['text'])[:2]).replace('\n',''), 1)

In [116]:
df_filter['coref_triples'] = df_filter['triple_text'].apply(lambda x: summarise_fast(x))

In [125]:
df_filter.head(10).coref_triples[5]

'BOOM! CHARLIE DANIELS Nails Obama And Democrats In Just One Tweet. Obama has been low key in the past few months even as he campaigned for a losing Hillary Clinton.Suddenly Obama and the Democrats decided Obama and the Democrats CARE about Russia and so Obama got all tough with Putin, which is sorta hilarious if you think about it. Dump on top of that the mess in Israel, Obamacare, the Iran fail, millions of Americans out of work, and the attempts at forcing states to fund Planned Parenthood, and you have a nice big MESS that Trump and Trump administration will have to figure out.Read more: Twitchy'

In [250]:
df_filter.to_csv('../../data/external/isot_dataset/fake_triples.csv')

In [243]:
## Isot-Dataset (True News)

In [176]:
df = pd.read_csv('../../data/external/isot_dataset/True.csv')

In [177]:
df[(df.text.str.contains('Michelle Obama')) & (df.text.str.contains('Trump'))]

,title,text,subject,date
3948,"Restaurant, retail calorie disclosure rule in ...",LOS ANGELES (Reuters) - A much-delayed U.S. ru...,politicsNews,"May 2, 2017"
5180,Really tremendous: Trump mimic Alec Baldwin co...,LOS ANGELES (Reuters) - Actor Alec Baldwin is ...,politicsNews,"March 2, 2017"
5496,"U.S., Japan first ladies: both unconventional ...",WASHINGTON (Reuters) - When Japanese first lad...,politicsNews,"February 10, 2017"
6182,Facts about the inauguration of Donald Trump a...,(Reuters) - With his left hand on a Bible used...,politicsNews,"January 19, 2017"
6214,Obama hopes for quiet after White House but wi...,WASHINGTON (Reuters) - He is ready for some qu...,politicsNews,"January 19, 2017"
6241,Ex-president George H.W. Bush moved to intensi...,(Reuters) - Former U.S. President George H.W. ...,politicsNews,"January 18, 2017"
6294,Martin Luther King's daughter says 'God can tr...,ATLANTA/NEW YORK (Reuters) - Martin Luther Kin...,politicsNews,"January 16, 2017"
6387,"Obama pushes values and prods Trump in final, ...","(The story refiles, to fix quote in 14th para...",politicsNews,"January 10, 2017"
6469,"In emotional farewell speech, Michelle Obama p...",WASHINGTON (Reuters) - First lady Michelle Oba...,politicsNews,"January 7, 2017"
6645,"Under threat in Washington, first lady's food ...",NEW YORK (Reuters) - Michelle Obama will be ce...,politicsNews,"December 18, 2016"


In [90]:
df_case = df[df.subject == 'politicsNews']

In [174]:
df_case['ym'] = df_case['date'].apply(lambda x: parser.parse(x)).astype(str)

In [229]:
df_case = df_case[df_case.ym.str[:7].isin(['2016-08','2016-09','2016-10','2016-11'])]

In [230]:
keys = ['election', 'trump', 'hillary', 'obama']

In [231]:
t_arrays = []
for i in keys:
    t_arrays.extend(np.where(df_case['text'].str.contains(i, case=False))[0])

In [232]:
df_filter = df_case.iloc[np.array(list(set(t_arrays)))]

In [233]:
len(df_filter)

1463

In [217]:
df_filter = df_filter.dropna(subset=['text', 'title'])

In [218]:
df_filter['fake'] = 0

In [519]:
df_filter = df_filter.reset_index()

In [508]:
df_filter['triple_text'] = df_filter[['title', 'text']].apply(lambda x: str(x['title'])+". "+" ".join(sent_tokenize(x['text'])[:2]).replace('\n',''), 1)

In [509]:
df_filter['coref_triples'] = df_filter['triple_text'].apply(lambda x: summarise_fast(x))

In [520]:
df_filter.to_csv('../../data/external/isot_dataset/real_triples.csv')

In [61]:
1428+1463

2891

In [559]:
### Combine Triples

In [593]:
fake_triples = pd.read_csv(
    '../../data/processed/reference_fake/reference_fake.txt', header=None, sep="\t"
)

In [594]:
real_triples = pd.read_csv(
    '../../data/processed/reference_true/reference_true.txt', header=None, sep="\t"
)

In [595]:
combined_triples = fake_triples.append(real_triples)

In [596]:
len(combined_triples)

45128

In [597]:
combined_triples.head()

,0
0,fake|0|Muslims | Stole Millions In | Gov ' t B...
1,fake|0|Muslims | Stole | Millions
2,fake|1|2016-10-28 | Print | administration
3,fake|2|HILLARY | GOES | OFF RAILS
4,fake|3|journalist | has | cell phone calls


In [598]:
comb = pd.read_csv(
    '../../data/processed/reference_model/reference_model.txt', header=None, sep="\t"
)

In [599]:
len(comb)

45128

In [601]:
comb.tail()

,0
45123,fake|2848|Temple Mount | is | Islamic site called
45124,fake|2848|Temple Mount | is | site called
45125,fake|2848|so-called Temple Mount | is | Islami...
45126,fake|2848|United Nations Educational | has pas...
45127,fake|2848|so-called Temple Mount | is | site


In [105]:
from nltk import WordNetLemmatizer

In [106]:
lemmatizer = WordNetLemmatizer()

In [108]:
lemmatizer.lemmatize('Vowed', 'v')

'Vowed'

In [207]:
true_triples = pd.read_csv(
    '../../data/processed/reference_model_t/reference_model_t.txt', header=None, sep="\t"
)

In [211]:
true_triples = pd.DataFrame(
    true_triples[0].str.split("|", 4).tolist(),
    columns=["status", "article_id", "e1", "r", "e2"],
)

In [209]:
from collections import Counter

In [ ]:
true_t

In [212]:
a = Counter(true_triples['e1'])

In [125]:
fake_triples.groupby('status').count()

,article_id,e1,r,e2
status,,,,
fake,17743,17743,17743,17743
true,27385,27385,27385,27385


In [5]:
### Swedish data ####

In [164]:
import os, json

In [167]:
json_dir = '../../data/external/swedish_news/'

In [172]:
files = [i for i in os.listdir(json_dir) if '.json' in i]

In [175]:
def read_json(json_path):
    with open(json_path) as f:
        content = json.load(f)
        d = {k:v for k,v in content.items() if k in ['uuid', 'title', 'text', 'published']}
        d['source'] = content['thread']['site']
        d['fake'] = 1
    return d

In [176]:
groupdf1 = [pd.Series(read_json(json_dir+files[i])) for i in range(len(files)) if i != 34952]
df = pd.concat(groupdf1, axis=1).transpose()

In [177]:
df.head()

,uuid,title,text,published,source,fake
0,72a45d2258f57ecb67797eef521861766253f01e,HV71 utsatt för fräckt inbrott,Senaste tidningen Plusklubben Viktigt! 2016-02...,2016-10-24T00:40:00.000+03:00,jp.se,1
1,e6e0fa043210a9856646cc89b07d97348746867e,Fyra fordon i brand i Borås,Fyra fordon i brand i Borås TT Fyra fordon bra...,2016-10-13T10:52:00.000+03:00,metro.se,1
2,7c018d132f99d6a4a7ed3255d092e8f3f780b5dd,Kjell-Olof Feldt: ”Jag upptäckte whisky – då s...,Kjell-Olof Feldt: ”Jag upptäckte whisky – då s...,2016-10-31T17:15:00.000+02:00,svt.se,1
3,f5f8eacc25eba43e2250a3092a4468abbf3eea9b,Eskilstuna-Kuriren – Inga vänner av tryckfrihet,Du är här: Hem » Politisk beteckning » Liberal...,2016-10-08T05:47:00.000+03:00,ekuriren.se,1
4,48c5ef1b3af74e1026147b8e13f0291c5d397118,"En enda regndag i oktober, extremtorrt och mån...","En enda regndag i oktober, extremtorrt och mån...",2016-10-16T03:00:00.000+03:00,pressen.se,1


In [178]:
keys = ['trump', 'hillary', 'election', 'obama']

In [179]:
t_arrays = []
for i in keys:
    t_arrays.extend(np.where(df['title'].str.contains(i, case=False))[0])

In [180]:
df_filter = df.iloc[np.array(list(set(t_arrays)))]

In [181]:
len(df_filter)

2819

In [182]:
len(set(df_filter.source))

133

In [194]:
len(df_filter)

2819

In [201]:
from sklearn.model_selection import train_test_split

In [207]:
small_df = df_filter

In [208]:
small_df['triple_text'] = small_df[['title', 'text']].apply(lambda x: str(x['title'])+". "+" ".join(sent_tokenize(x['text'])[:2]).replace('\n',''), 1)

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [209]:
small_df.head(10)['triple_text'].iloc[1]

'Två kvinnor berättar: ”Trump tafsade på oss”. Stefan Franzén stefan.franzen@nsk.se Två kvinnor har trätt fram i New York Times och berättat att presidentkandidaten Donald Trump tafsat på dem och rört vid dem på ett olämpligt sätt. Den ena händelsen ska ha ägt rum i början av 1980-talet medan den an dra ägde rum 2005.'

In [210]:
import copy
import time
from googletrans import Translator

translatedList = []
counter = 0
for index, row in small_df.iterrows():
    # REINITIALIZE THE API
    counter += 1
    print(counter)
    time.sleep(5)
    translator = Translator()
    newrow = copy.deepcopy(row)
    try:
        # translate the 'text' column
        translated = translator.translate(row['triple_text'], src='sv', dest='en')
        newrow['triple_translated'] = translated.text
    except Exception as e:
        print(str(e))
        break
    translatedList.append(newrow)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [211]:
df_trips = pd.DataFrame(translatedList)

In [212]:
df_trips.head()['triple_translated']

16385     Trump paused in Finnish miss. Trump paused in ...
196613    Two women say, "Trump paused on us". Stefan Fr...
122886    Recording shows Trump's women's account. Recor...
221190    TV host shuts off after Trumpskandal. TV host ...
106504    Get to know the candidate: Hillary Clinton. Do...
Name: triple_translated, dtype: object

In [213]:
df_trips['coref_triples'] = df_trips['triple_translated'].apply(lambda x: summarise_fast(x))

In [214]:
df_trips.head()

,uuid,title,text,published,source,fake,triple_text,triple_translated,coref_triples
16385,fdcc1b252bc62bff10a49c8fb632d1077d7d8fae,Trump tafsade på finsk miss,Trump tafsade på finsk miss TT-Utrikes 17 minu...,2016-10-28T05:09:00.000+03:00,bt.se,1,Trump tafsade på finsk miss. Trump tafsade på ...,Trump paused in Finnish miss. Trump paused in ...,Trump paused in Finnish miss. Trump paused in ...
196613,7927a619d0eed251d266046685fd98870328b7b7,Två kvinnor berättar: ”Trump tafsade på oss”,Stefan Franzén stefan.franzen@nsk.se Två kvinn...,2016-10-13T12:52:00.000+03:00,laholmstidning.se,1,Två kvinnor berättar: ”Trump tafsade på oss”. ...,"Two women say, ""Trump paused on us"". Stefan Fr...","Two women say, ""Trump paTwo womened on Two wom..."
122886,ebff5712b2ebed74b70f5a7c6a2e9ed4d47fc64c,Inspelning visar Trumps kvinnoförakt,Inspelning visar Trumps kvinnoförakt Publicera...,2016-10-08T07:00:00.000+03:00,sverigesradio.se,1,Inspelning visar Trumps kvinnoförakt. Inspelni...,Recording shows Trump's women's account. Recor...,Recording shows Trump's women's account. Recor...
221190,de28fa7ab53702fe414fe765e20e757fbaa149a7,Tv-värd stängs av efter Trumpskandal,Tv-värd stängs av efter Trumpskandal Programle...,2016-10-09T03:00:00.000+03:00,pressen.se,1,Tv-värd stängs av efter Trumpskandal. Tv-värd ...,TV host shuts off after Trumpskandal. TV host ...,TV host shuts off after Trumpskandal. TV host ...
106504,089f94231389975d01a815ad76138586de942479,Lär känna kandidaten: Hillary Clinton,Inrikes Utrikes Ekonomi Vetenskap Ođđasat Uuti...,2016-10-26T22:45:00.000+03:00,svt.se,1,Lär känna kandidaten: Hillary Clinton. Inrikes...,Get to know the candidate: Hillary Clinton. Do...,Get to know the candidate: Hillary Clinton. Do...


In [215]:
df_trips.to_csv('../../data/external/swedish_news/triples_sample.csv')


In [8]:
df_trips = pd.read_csv('../../data/external/swedish_news/triples_sample.csv')

In [29]:
df_trips[df_trips.coref_triples.str.contains('Donald Trump was bowled')]

,Unnamed: 0,index,Unnamed: 0.1,uuid,title,text,published,source,fake,triple_text,triple_translated,coref_triples
218,218,218,229986,0f9cb93d59c5d73d0e926d9eb6151065d4cba6a1,Donald Trump buades ut vid middag,Donald Trump buades ut vid middag 12:16 | 16-1...,2016-10-21T18:16:00.000+03:00,unt.se,1,Donald Trump buades ut vid middag. Donald Trum...,Donald Trump was bowled out at noon. Donald Tr...,Donald Trump was bowled out at noon. Donald Tr...
341,341,341,17364,81084824bad2795ad092c0ebc1df3d0fba695176,Donald Trump buades ut vid middag,Donald Trump buades ut vid middag TT-Utrikes 5...,2016-10-21T11:33:00.000+03:00,ystadsallehanda.se,1,Donald Trump buades ut vid middag. Donald Trum...,Donald Trump was bowled out at noon. Donald Tr...,Donald Trump was bowled out at noon. Donald Tr...
366,366,366,9272,e2b41d4a6adc6d5274ae275f2ff22b50ddc4701e,Donald Trump buades ut vid middag,Donald Trump buades ut vid middag 12:16 | 16-1...,2016-10-21T18:16:00.000+03:00,corren.se,1,Donald Trump buades ut vid middag. Donald Trum...,Donald Trump was bowled out at noon. Donald Tr...,Donald Trump was bowled out at noon. Donald Tr...
457,457,457,42284,ac7bd4bb4ae03e02cb17ac0d62a1da40f7ef96d3,Donald Trump buades ut vid middag,,2016-10-21T18:16:00.000+03:00,kuriren.nu,1,Donald Trump buades ut vid middag.,Donald Trump was bowled out at noon.,Donald Trump was bowled out at noon.
525,525,525,206309,a570c2bbe310d16c0e94729b42f600d4f7dc06fc,Donald Trump buades ut vid middag,Donald Trump buades ut vid middag TT-Utrikes 1...,2016-10-21T11:33:00.000+03:00,barometern.se,1,Donald Trump buades ut vid middag. Donald Trum...,Donald Trump was bowled out at noon. Donald Tr...,Donald Trump was bowled out at noon. Donald Tr...
631,631,631,222975,aa054a71e6764de42c76f06ab9811b25646919b9,Donald Trump buades ut vid middag,Donald Trump buades ut vid middag 05:33 | 16-1...,2016-10-21T11:33:00.000+03:00,unt.se,1,Donald Trump buades ut vid middag. Donald Trum...,Donald Trump was bowled out at noon. Donald Tr...,Donald Trump was bowled out at noon. Donald Tr...
655,655,655,182087,bfa2b3203d1bf9a634196f842d9160f3b0af2dd6,Donald Trump buades ut vid middag,Donald Trump buades ut vid middag TT-Utrikes 7...,2016-10-21T11:33:00.000+03:00,sydostran.se,1,Donald Trump buades ut vid middag. Donald Trum...,Donald Trump was bowled out at noon. Donald Tr...,Donald Trump was bowled out at noon. Donald Tr...
672,672,672,190320,ffe15bb13d2227b1800e5642ef634f0a122b9664,Donald Trump buades ut vid middag,Donald Trump buades ut vid middag TT-Utrikes 5...,2016-10-21T12:19:00.000+03:00,trelleborgsallehanda.se,1,Donald Trump buades ut vid middag. Donald Trum...,Donald Trump was bowled out at noon. Donald Tr...,Donald Trump was bowled out at noon. Donald Tr...
869,869,869,174527,e64921842e3d2158f39895d93f8a3eeaf45170fd,Donald Trump buades ut vid middag,Donald Trump buades ut vid middag Hillary Clin...,2016-10-21T12:01:00.000+03:00,svt.se,1,Donald Trump buades ut vid middag. Donald Trum...,Donald Trump was bowled out at noon. Donald Tr...,Donald Trump was bowled out at noon. Donald Tr...
874,874,874,158151,e47d782bd17a5240bca8fb14cd616c619546b27d,Donald Trump buades ut vid middag,Donald Trump buades ut vid middag Donald Trump...,2016-10-21T12:09:00.000+03:00,di.se,1,Donald Trump buades ut vid middag. Donald Trum...,Donald Trump was bowled out at noon. Donald Tr...,Donald Trump was bowled out at noon. Donald Tr...


In [10]:
len(df_trips)

2819

In [96]:
df_trips.head(10)['triple_translated'].iloc[6]

'Michael Moore releases Trump movie. Michael Moore releases Trump film The Oscar-winning "Bowling for Columbine" director Michael Moore has secretly worked on a documentary about the US presidential candidate Donald Trump. (TT).'

In [154]:
import os

In [217]:
files = os.listdir('../../data/processed/cv-models/swedish_data_v3/small_files')

In [218]:
for i in files:
    os.rename('../../data/processed/cv-models/swedish_data_v3/small_files/'+i, '../../data/processed/cv-models/swedish_data_v3/small_files/swedish_'+i)